In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import SimpleITK as sitk
import nibabel as nib
from monai.transforms import Resize
import time
import gc
import tracemalloc

from matplotlib import figure

In [2]:
IMG_BASE_DIR = '/workspace/BoneMeta_new'
LABEL_BASE_DIR = '/workspace/BoneMeta_raw_02'
SAVE_BASE_DIR = '/workspace'
IMG_FOLDER_NAME = 'images'
LABEL_FOLDER_NAME = 'label'

WINDOWS = [(500,200), (1200,400)]
DATA_INFO_PATH = '/workspace/DataInfo/data_info_20220310_VI.csv'

In [3]:
DATA_INFO = pd.read_csv(DATA_INFO_PATH)
DATA_INFO = DATA_INFO.sort_values(['Case', 'File Name'])
DATA_INFO.reset_index(drop=True, inplace=True)
DATA_INFO.set_index(['Case','File Name'], inplace=True)

IMG_DIR = os.path.join(IMG_BASE_DIR, IMG_FOLDER_NAME)
LABEL_DIR = os.path.join(LABEL_BASE_DIR, LABEL_FOLDER_NAME)

IMG_FILES = os.listdir(IMG_DIR)
LABEL_FILES = os.listdir(LABEL_DIR)

def get_img_path(file): 
    return os.path.join(IMG_DIR, file)

def get_label_path(case, file):
    return os.path.join(LABEL_DIR, case, file)

def case_to_file(case):
    return case+'.npy'

def file_to_case(file_name):
    return file_name.split('.')[0]

In [4]:
DATA_INFO.head()

Data  \
Case                                               File Name                             
000002_20180829_Chest_CT_(contrast)                lesionAnnot3D-001.nii.gz       test   
                                                   lesionAnnot3D-004.nii.gz       test   
000005_20181202_CT_Abdomen+Pelvis_Dynamic_(cont... lesionAnnot3D-001.nii.gz  train_add   
000009_20180417_Chest_CT_(contrast)                lesionAnnot3D-001.nii.gz       test   
                                                   lesionAnnot3D-002.nii.gz       test   

                                                                            Lesion Type  \
Case                                               File Name                              
000002_20180829_Chest_CT_(contrast)                lesionAnnot3D-001.nii.gz           S   
                                                   lesionAnnot3D-004.nii.gz           S   
000005_20181202_CT_Abdomen+Pelvis_Dynamic_(cont... lesionAnnot3D-001.nii.gz           L   
000009_20180417_Chest_CT_(contrast)                lesionAnnot3D-001.nii.gz           S   
                                                   lesionAnnot3D-002.nii.gz           S   

                                                                            Lesion Location  \
Case                                               File Name                                  
000002_20180829_Chest_CT_(contrast)                lesionAnnot3D-001.nii.gz              C7   
                                                   lesionAnnot3D-004.nii.gz             T10   
000005_20181202_CT_Abdomen+Pelvis_Dynamic_(cont... lesionAnnot3D-001.nii.gz              L5   
000009_20180417_Chest_CT_(contrast)                lesionAnnot3D-001.nii.gz            C7-1   
                                                   lesionAnnot3D-002.nii.gz            C7-2   

                                                                            I/V  \
Case                                               File Name                      
000002_20180829_Chest_CT_(contrast)                lesionAnnot3D-001.nii.gz   I   
                                                   lesionAnnot3D-004.nii.gz   I   
000005_20181202_CT_Abdomen+Pelvis_Dynamic_(cont... lesionAnnot3D-001.nii.gz   V   
000009_20180417_Chest_CT_(contrast)                lesionAnnot3D-001.nii.gz   V   
                                                   lesionAnnot3D-002.nii.gz   I   

                                                                             Lesion Size  \
Case                                               File Name                               
000002_20180829_Chest_CT_(contrast)                lesionAnnot3D-001.nii.gz            0   
                                                   lesionAnnot3D-004.nii.gz            0   
000005_20181202_CT_Abdomen+Pelvis_Dynamic_(cont... lesionAnnot3D-001.nii.gz            0   
000009_20180417_Chest_CT_(contrast)                lesionAnnot3D-001.nii.gz            0   
                                                   lesionAnnot3D-002.nii.gz            0   

                                                                             Background Size  \
Case                                               File Name                                   
000002_20180829_Chest_CT_(contrast)                lesionAnnot3D-001.nii.gz            13125   
                                                   lesionAnnot3D-004.nii.gz            14500   
000005_20181202_CT_Abdomen+Pelvis_Dynamic_(cont... lesionAnnot3D-001.nii.gz           198880   
000009_20180417_Chest_CT_(contrast)                lesionAnnot3D-001.nii.gz            11136   
                                                   lesionAnnot3D-002.nii.gz             1820   

                                                                              Original Shape  \
Case                                               File Name                                   
000002_20180829_Chest_CT_(contrast)                lesi

In [5]:
def get_image_file(filename):
    return np.load(get_img_path(filename))

def get_label_file(case, filename):
    label = nib.load(get_label_path(case, filename))
    label_npy = label.get_fdata().transpose((2,1,0))
    return label_npy.astype(np.uint8)

In [6]:
print(IMG_DIR, len(IMG_FILES))

/workspace/BoneMeta_new/images 402


In [7]:
# 케이스별 DICOM 읽기: case_name, base_dir -> npy
def dcm_to_numpy(case_name, base_dir):
  dicoms = sorted(glob.glob(f'{base_dir}/{case_name}/*'))
  dcm_reader = sitk.ImageSeriesReader()
  dcm_reader.SetFileNames(dicoms)
  img_arr = sitk.GetArrayFromImage(dcm_reader.Execute())
  return img_arr

def slice_img(img, offset, end, interval):
  if end is None:
    end = len(img)
  else: 
    if end > len(img): 
      end = len(img)
    if end <= offset:
      end = offset + 1
  return img[offset:end:interval]

def show_numpy_img(np_img, offset=0, end=None, interval=5, title=''): 
  sliced_img = slice_img(np_img, offset, end, interval)

  figsize_per_img = 3
  num_col = 5
  num_row = int(np.ceil(sliced_img.shape[0] / num_col))
  # fig, axs = plt.subplots(num_row, num_col, figsize = (figsize_per_img*num_col, figsize_per_img*num_row))
  plt.figure(figsize=(figsize_per_img*num_col, figsize_per_img*num_row))
  for i, img in enumerate(sliced_img):
    if i >= num_col*num_row:
      continue
    # axs[i].imshow(img)
    plt.subplot(num_row, num_col, i+1)
    plt.imshow(img, 'gray')
    # plt.title()
  plt.suptitle(title)
  plt.tight_layout()
  plt.show()

# npy image windowing 하기
def adjust_window(image, window):
    width = window[0]
    level = window[1]
    upper = level+width/2
    lower = level-width/2
    copied_image = image.copy()
    copied_image[copied_image<lower] = lower
    copied_image[copied_image>upper] = upper
    copied_image = copied_image-lower
    return (copied_image/(upper-lower)*255).round().astype(np.uint8)

# # 2nd version
# def adjust_window(image, window):
#     width = window[0]
#     level = window[1]
#     upper = level+width/2
#     lower = level-width/2
#     copied_image = image.clip(lower, upper)
#     copied_image = copied_image-lower
#     return (copied_image/(upper-lower)*255)

#npy image 정보 보여주기 
def show_img_info(np_img):
  print('Shape:', np_img.shape)
  print('DType:', np_img.dtype)
  print('Max:', np_img.max())
  print('Min:', np_img.min())
  print(np_img)

In [8]:
# label있는 부분만 저장
def save_img_and_lesion_label(np_img, label, case, lesion_filename, tag ='', interval = 2, figsize_per_image = 10):
  offset = 0
  end = None
  sliced_img = slice_img(np_img, offset, end, interval)
  sliced_label = slice_img(label, offset, end, interval)
  
  label_idx = sliced_label.any(axis=(1,2))
  sliced_img = sliced_img[label_idx]
  sliced_label = sliced_label[label_idx]

  num_row = len(sliced_img)
  num_col = 2

  save_dir = f'{SAVE_BASE_DIR}/image_check/{case}'
  os.makedirs(save_dir, exist_ok=True)

  for i in range(0, num_row):
    save_path = f'{save_dir}/{case[:30]}_{file_to_case(lesion_filename)}_{tag}_{i}.png'
    if os.path.exists(save_path): 
      continue;
    f = figure.Figure(figsize=(figsize_per_image*num_col, figsize_per_image))
    f.patch.set_facecolor('black')
    plt.subplot(1, num_col, 1)
    tissue_image = sliced_img[i]
    plt.imshow(tissue_image, 'gray')

    plt.subplot(1, num_col, 2)
    mask = sliced_label[i]
    label_on_tissue = sitk.LabelMapContourOverlay(sitk.Cast(sitk.GetImageFromArray(mask), sitk.sitkLabelUInt8), sitk.GetImageFromArray(tissue_image), opacity=0.7, contourThickness=[2,2], colormap=(0,255,0))
    plt.imshow(sitk.GetArrayFromImage(label_on_tissue), 'gray')

    plt.tight_layout()
    plt.savefig(save_path)
    
#     f.clf()
#   plt.show()
#     plt.close('all')
    plt.close(f)
    plt.clf()

Pseudocode
```
DATA_INFO 기준으로 진행. 
for case in DATA_INFO
  read img
  iso_voxel img
  for lesion in DATA_INFO, case
    read label 
    iso_vosel label 
    
    join and save img and label 
     - axial sagittal 
     - (500,200), (1200,400)

```

In [9]:
WINDOWS = [(500,200), (1200,400)]
def resize(npy, resizer):
    return resizer(npy[np.newaxis,])[0,:]

In [10]:
def save_lesion_img_from_data_info(data_info):
    working_case = ''
    for i, (case, lesion_file) in enumerate(data_info.index.values):
        try:             
            if working_case != case:
                working_case = case
                img = get_image_file(case_to_file(case))

                xyz_string = DATA_INFO.loc[(case, lesion_file), 'Original Shape'][0]
                x_spacing = DATA_INFO.loc[(case, lesion_file), 'X Spacing'][0]
                y_spacing = DATA_INFO.loc[(case, lesion_file), 'Y Spacing'][0]
                z_spacing = abs(DATA_INFO.loc[(case, lesion_file), 'Slice Interval'][0])
                xyz = xyz_string[1:-1].split(', ')
                x,y,z = [int(i) for i in xyz]
                new_shape = [int(x * x_spacing), int(y * y_spacing), int(z * z_spacing)]
                resizer = Resize(new_shape)

                img = resize(img, resizer)

            label = get_label_file(case, lesion_file)
            label = resize(label, resizer)
            
            iv = DATA_INFO.loc[(case, lesion_file),'I/V'][0]
            slm_type = DATA_INFO.loc[(case, lesion_file),'Lesion Type'][0]
            
            for window in WINDOWS: 
                adj_image = adjust_window(img, window)
                save_img_and_lesion_label(adj_image, label, case, lesion_file, tag = 'Axial'+str(window)+f'_{iv}_{slm_type}')

                coronal_image = adj_image.transpose(1,0,2)
                coronal_label = label.transpose(1,0,2)
                save_img_and_lesion_label(coronal_image, coronal_label, case, lesion_file, tag = 'Cor'+str(window)+f'_{iv}_{slm_type}')

            gc.collect()
        except:
            print(case, lesion_file)
            continue;

In [12]:
class Timer():
    start = 0
    end = 0
    def start(self):
        self.start = time.perf_counter()
        
    def end(self):
        self.end = time.perf_counter()
        print(self.end - self.start)
        return(self.end - self.start)

In [13]:
from multiprocessing import Pool

timer = Timer()
timer.start()
num_process = 45
chunk_size = len(DATA_INFO) // num_process + 1

datainfo_list = [DATA_INFO.iloc[i*chunk_size: (i+1)*chunk_size] for i in range(num_process)]
with Pool(processes=num_process) as pool:
    
    pool.map(save_lesion_img_from_data_info, datainfo_list)
    pool.close() 
    pool.join()
timer.end()

<ipython-input-7-afb7fca23977>:48: size=32.1 MiB, count=2, average=16.1 MiB
<frozen importlib._bootstrap_external>:487: size=194 KiB, count=2447, average=81 B
/usr/lib/python3.6/multiprocessing/queues.py:337: size=127 KiB, count=1538, average=85 B
/usr/local/lib/python3.6/dist-packages/matplotlib/transforms.py:201: size=38.4 KiB, count=8, average=4912 B
/usr/lib/python3.6/_weakrefset.py:84: size=37.5 KiB, count=151, average=255 B
memory_blocks= 2 size_kB= 32893.2763671875
  File "<ipython-input-7-afb7fca23977>", line 48
    return (copied_image/(upper-lower)*255).round().astype(np.uint8)
<ipython-input-7-afb7fca23977>:48: size=46.1 MiB, count=2, average=23.0 MiB
<frozen importlib._bootstrap_external>:487: size=193 KiB, count=2430, average=81 B
/usr/lib/python3.6/multiprocessing/queues.py:337: size=127 KiB, count=1522, average=85 B
/usr/local/lib/python3.6/dist-packages/matplotlib/transforms.py:201: size=38.4 KiB, count=8, average=4912 B
/usr/lib/python3.6/_weakrefset.py:84: size=37.6 K

/usr/local/lib/python3.6/dist-packages/matplotlib/transforms.py:201: size=38.4 KiB, count=8, average=4912 B
/usr/lib/python3.6/_weakrefset.py:84: size=38.3 KiB, count=161, average=244 B
memory_blocks= 2 size_kB= 26802.9169921875
  File "<ipython-input-7-afb7fca23977>", line 48
    return (copied_image/(upper-lower)*255).round().astype(np.uint8)
<ipython-input-7-afb7fca23977>:48: size=39.7 MiB, count=2, average=19.9 MiB
<frozen importlib._bootstrap_external>:487: size=192 KiB, count=2410, average=81 B
/usr/lib/python3.6/multiprocessing/queues.py:337: size=128 KiB, count=1548, average=85 B
/usr/lib/python3.6/_weakrefset.py:84: size=38.5 KiB, count=164, average=241 B
/usr/local/lib/python3.6/dist-packages/matplotlib/transforms.py:201: size=38.4 KiB, count=8, average=4912 B
memory_blocks= 2 size_kB= 40686.65234375
  File "<ipython-input-7-afb7fca23977>", line 48
    return (copied_image/(upper-lower)*255).round().astype(np.uint8)
<ipython-input-7-afb7fca23977>:48: size=60.5 MiB, count=2, a

3258.382281505037